# Test full calculation
Test the full calculation of the 

## Import packages

In [1]:
%matplotlib inline
import xarray as xr
from xmip.preprocessing import rename_cmip6
import s3fs
import zarr

import util

In [2]:
from dask_gateway import Gateway
gateway = Gateway()

In [3]:
##A line of trick to clean your dask cluster before you start your computation
from dask.distributed import Client
clusters=gateway.list_clusters()
print(clusters )
for cluster in clusters :
    cluster= gateway.connect(cluster.name)
    print(cluster)
    client = Client(cluster)
    client.close()
    cluster.shutdown()

[ClusterReport<name=daskhub.9eb8947fcd38485194ede8d65303f6d7, status=RUNNING>]
GatewayCluster<daskhub.9eb8947fcd38485194ede8d65303f6d7, status=running>


/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:1014: RuntimeWarning: coroutine 'rpc.close_rpc' was never awaited
  self.scheduler_comm.close_rpc()


### Create a new Dask cluster with the Dask Gateway

In [4]:
cluster = gateway.new_cluster(worker_memory=2, worker_cores=1)

cluster.scale(20)
cluster

### Get a client from the Dask Gateway Cluster

As stated above, creating a Dask `Client` is mandatory in order to perform following Daks computations on your Dask Cluster.

In [5]:
from distributed import Client

if cluster:
    client = Client(cluster) # create a dask Gateway cluster
else:
    client = Client()   # create a local dask cluster on the machine.
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /jupyterhub/services/dask-gateway/clusters/daskhub.0b122d860e8c4f7d914d74570d68f751/status,


## Summary of some test data

| model name | resolution | years in file for one variable | file size (one variable) | combined 3D variable data size | combined 2D variable data size |
| --- | --- | --- | --- | --- | --- |
| ACCESS-OM2 | 100 km | 10 | 670 MB | 17 GB | 0.4 GB |
| ACCESS-OM2-025 | 25 km | 1 | 1.1 GB | 271 GB | 7 GB |
| CMCC-CM2-SR5 | 100 km | 61 | 3.6 GB | 15 GB | 0.4 GB |
| CMCC-CM2-HR4 | 25 km | 10 | 9.4 GB | 231 GB | 6 GB |

## Load data from ESGF

In [6]:
result = util.esgf_search(activity_id='OMIP'
                          , experiment_id='omip2'
                          , variable_id='vmo'
                          , source_id='ACCESS-OM2-025')
                          # works, source_id='ACCESS-OM2')
#CMCC-CM2-SR5')

https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=ACCESS-OM2-025&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=ACCESS-OM2-025&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=10
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=ACCESS-OM2-025&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=20
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=ACCESS-OM2-025&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=30
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=ACCESS-OM2-025&project=CMIP6&type=File&distrib=false&format=application%

In [7]:
# pick one file for testing
files_to_open = [result[-1]]

In [8]:
ds = xr.open_mfdataset(files_to_open, combine='by_coords',chunks={'j':150,'i':150})

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/coding/times.py:682: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


In [9]:
ds=ds.persist()
ds

<xarray.Dataset>
Dimensions:             (time: 12, bnds: 2, lev: 50, j: 1080, i: 1440,
                         vertices: 4)
Coordinates:
  * time                (time) object 0366-01-16 12:00:00 ... 0366-12-16 12:0...
  * lev                 (lev) float64 1.152 3.649 6.565 ... 5.034e+03 5.254e+03
  * j                   (j) int32 0 1 2 3 4 5 ... 1074 1075 1076 1077 1078 1079
  * i                   (i) int32 0 1 2 3 4 5 ... 1434 1435 1436 1437 1438 1439
    latitude            (j, i) float64 dask.array<chunksize=(150, 150), meta=np.ndarray>
    longitude           (j, i) float64 dask.array<chunksize=(150, 150), meta=np.ndarray>
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (time, bnds) object dask.array<chunksize=(12, 2), meta=np.ndarray>
    lev_bnds            (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float64 dask.array<chunksize=(150, 150, 4), meta=np.ndarray>
    vertices_longitude  (j, i, vertices) float64 dask.array<chunksize=(150, 150, 4), meta=np.ndarray>
    vmo                 (time, lev, j, i) float32 dask.array<chunksize=(12, 50, 150, 150), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     OMIP
    branch_method:                   no parent
    branch_time_in_child:            0.0
    branch_time_in_parent:           0.0
    creation_date:                   2021-06-17T23:34:13Z
    ...                              ...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    version:                         v20210617
    license:                         CMIP6 model data produced by CSIRO is li...
    cmor_version:                    3.4.0
    DODS_EXTRA.Unlimited_Dimension:  time

## Pre-proc with `XMIP`

In [10]:
ds = rename_cmip6(ds)

In [11]:
ds

<xarray.Dataset>
Dimensions:      (time: 12, bnds: 2, lev: 50, y: 1080, x: 1440, vertex: 4)
Coordinates:
  * time         (time) object 0366-01-16 12:00:00 ... 0366-12-16 12:00:00
  * lev          (lev) float64 1.152 3.649 6.565 ... 5.034e+03 5.254e+03
  * y            (y) int32 0 1 2 3 4 5 6 ... 1073 1074 1075 1076 1077 1078 1079
  * x            (x) int32 0 1 2 3 4 5 6 ... 1433 1434 1435 1436 1437 1438 1439
Dimensions without coordinates: bnds, vertex
Data variables:
    time_bounds  (time, bnds) object dask.array<chunksize=(12, 2), meta=np.ndarray>
    lev_bounds   (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    lat_bounds   (y, x, vertex) float64 dask.array<chunksize=(150, 150, 4), meta=np.ndarray>
    lon_bounds   (y, x, vertex) float64 dask.array<chunksize=(150, 150, 4), meta=np.ndarray>
    lat          (y, x) float64 dask.array<chunksize=(150, 150), meta=np.ndarray>
    lon          (y, x) float64 dask.array<chunksize=(150, 150), meta=np.ndarray>
    vmo          (time, lev, y, x) float32 dask.array<chunksize=(12, 50, 150, 150), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     OMIP
    branch_method:                   no parent
    branch_time_in_child:            0.0
    branch_time_in_parent:           0.0
    creation_date:                   2021-06-17T23:34:13Z
    ...                              ...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    version:                         v20210617
    license:                         CMIP6 model data produced by CSIRO is li...
    cmor_version:                    3.4.0
    DODS_EXTRA.Unlimited_Dimension:  time

In [12]:
def add_geo(DS):
    # assign geolat and geolon from ocean static
    DS = DS.assign_coords({'lon':DS['lon'].sel(x=DS.x,y=DS.y),
                           'lat':DS['lat'].sel(x=DS.x,y=DS.y)})
    return DS

In [13]:
ds = add_geo(ds)

In [14]:
ds

<xarray.Dataset>
Dimensions:      (time: 12, bnds: 2, lev: 50, y: 1080, x: 1440, vertex: 4)
Coordinates:
  * time         (time) object 0366-01-16 12:00:00 ... 0366-12-16 12:00:00
  * lev          (lev) float64 1.152 3.649 6.565 ... 5.034e+03 5.254e+03
  * y            (y) int32 0 1 2 3 4 5 6 ... 1073 1074 1075 1076 1077 1078 1079
  * x            (x) int32 0 1 2 3 4 5 6 ... 1433 1434 1435 1436 1437 1438 1439
    lat          (y, x) float64 dask.array<chunksize=(150, 150), meta=np.ndarray>
    lon          (y, x) float64 dask.array<chunksize=(150, 150), meta=np.ndarray>
Dimensions without coordinates: bnds, vertex
Data variables:
    time_bounds  (time, bnds) object dask.array<chunksize=(12, 2), meta=np.ndarray>
    lev_bounds   (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    lat_bounds   (y, x, vertex) float64 dask.array<chunksize=(150, 150, 4), meta=np.ndarray>
    lon_bounds   (y, x, vertex) float64 dask.array<chunksize=(150, 150, 4), meta=np.ndarray>
    vmo          (time, lev, y, x) float32 dask.array<chunksize=(12, 50, 150, 150), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     OMIP
    branch_method:                   no parent
    branch_time_in_child:            0.0
    branch_time_in_parent:           0.0
    creation_date:                   2021-06-17T23:34:13Z
    ...                              ...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    version:                         v20210617
    license:                         CMIP6 model data produced by CSIRO is li...
    cmor_version:                    3.4.0
    DODS_EXTRA.Unlimited_Dimension:  time

## Subset Pan-Arctic (lat>50N)

In [15]:
cond = (ds['lat']>=50)
dsnow = ds[['vmo']].where(cond,drop=True).persist()

In [16]:
dsnow = dsnow.chunk(chunks={'x':150,'y':150}).persist()

In [17]:
dsnow

<xarray.Dataset>
Dimensions:  (time: 12, lev: 50, y: 350, x: 1440)
Coordinates:
  * time     (time) object 0366-01-16 12:00:00 ... 0366-12-16 12:00:00
  * lev      (lev) float64 1.152 3.649 6.565 ... 4.815e+03 5.034e+03 5.254e+03
  * y        (y) int32 730 731 732 733 734 735 ... 1074 1075 1076 1077 1078 1079
  * x        (x) int32 0 1 2 3 4 5 6 7 ... 1433 1434 1435 1436 1437 1438 1439
    lat      (y, x) float64 dask.array<chunksize=(150, 150), meta=np.ndarray>
    lon      (y, x) float64 dask.array<chunksize=(150, 150), meta=np.ndarray>
Data variables:
    vmo      (time, lev, y, x) float32 dask.array<chunksize=(12, 50, 150, 150), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     OMIP
    branch_method:                   no parent
    branch_time_in_child:            0.0
    branch_time_in_parent:           0.0
    creation_date:                   2021-06-17T23:34:13Z
    ...                              ...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    version:                         v20210617
    license:                         CMIP6 model data produced by CSIRO is li...
    cmor_version:                    3.4.0
    DODS_EXTRA.Unlimited_Dimension:  time

In [18]:
your_name='tinaok'
path='WAFFLES/'+your_name
s3_prefix =  "s3://"+path
print(s3_prefix)
access_key = !aws configure get aws_access_key_id
access_key = access_key[0]
secret_key = !aws configure get aws_secret_access_key
secret_key = secret_key[0]
client_kwargs={'endpoint_url': 'https://object-store.cloud.muni.cz'}

s3://WAFFLES/tinaok


In [19]:
target = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://object-store.cloud.muni.cz'
      })
target.ls(path)

['WAFFLES/tinaok/FSStore_zarr',
 'WAFFLES/tinaok/FSStore_zarr_ACCESS-OM2-025',
 'WAFFLES/tinaok/test_ACCESS-OM2-025']

In [20]:
target.rm(path,recursive=True)

In [21]:
zarr_file_name= "test_ACCESS-OM2-025"
uri = f"{s3_prefix}/{zarr_file_name}"
store = zarr.storage.FSStore(uri, client_kwargs=client_kwargs,key=access_key, secret=secret_key)
store.map.fs.find(uri)

[]

In [22]:
%time dsnow.to_zarr(store=store, mode='w', consolidated=True)

CPU times: user 1.15 s, sys: 149 ms, total: 1.3 s
Wall time: 1min 23s


In [23]:
test=xr.open_zarr(store=store)#consolidated=True)
test

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/coding/times.py:682: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


<xarray.Dataset>
Dimensions:  (y: 350, x: 1440, lev: 50, time: 12)
Coordinates:
    lat      (y, x) float64 dask.array<chunksize=(150, 150), meta=np.ndarray>
  * lev      (lev) float64 1.152 3.649 6.565 ... 4.815e+03 5.034e+03 5.254e+03
    lon      (y, x) float64 dask.array<chunksize=(150, 150), meta=np.ndarray>
  * time     (time) object 0366-01-16 12:00:00 ... 0366-12-16 12:00:00
  * x        (x) int32 0 1 2 3 4 5 6 7 ... 1433 1434 1435 1436 1437 1438 1439
  * y        (y) int32 730 731 732 733 734 735 ... 1074 1075 1076 1077 1078 1079
Data variables:
    vmo      (time, lev, y, x) float32 dask.array<chunksize=(12, 50, 150, 150), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                     CF-1.7 CMIP-6.2
    DODS_EXTRA.Unlimited_Dimension:  time
    activity_id:                     OMIP
    branch_method:                   no parent
    branch_time_in_child:            0.0
    branch_time_in_parent:           0.0
    ...                              ...
    table_info:                      Creation Date:(30 April 2019) MD5:12ec21...
    title:                           ACCESS-OM2-025 output prepared for CMIP6
    tracking_id:                     hdl:21.14100/d0cba987-c5c7-42fc-a310-ae1...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    version:                         v20210617

In [26]:
test.y.compute()

<xarray.DataArray 'y' (y: 350)>
array([ 730,  731,  732, ..., 1077, 1078, 1079], dtype=int32)
Coordinates:
  * y        (y) int32 730 731 732 733 734 735 ... 1074 1075 1076 1077 1078 1079
Attributes:
    long_name:  cell index along second dimension
    units:      1

In [28]:
import hvplot
import hvplot.xarray
import cartopy.crs as ccrs
import geoviews.feature as gf
import cartopy.feature as cfeature
plot=test.hvplot.quadmesh( x='lon', y='lat'
                          #,widget_location='bottom'
                          #,widget_location='left' # bugs h
                          #,widgets={'t': pn.widgets.DiscreteSlider}
                          #,widget_type='scrubber'
                          , rasterize=True
                          , geo=True
                         #,global_extent=True
                          ,projection=ccrs.Orthographic(-60, 90) #
                         # ,projection=ccrs.NorthPolarStereo()    #
                          , project=True
                         # , cmap=cmap
                         # ,clim=clim
                         # ,clabel=clabel
                           , cmap='rainbow'
                         # ,width=350, height=300 #find best size
                          # ,title=filename.replace('_', ' ')
                            )
plot =  gf.land * plot * gf.coastline * gf.grid
plot

/srv/conda/envs/notebook/lib/python3.9/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_physical/ne_110m_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/srv/conda/envs/notebook/lib/python3.9/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_physical/ne_110m_graticules_30.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/srv/conda/envs/notebook/lib/python3.9/site-packages/geoviews/operation/projection.py:99: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if proj_geom.geom_type == 'GeometryCollection' and len(proj_geom) == 0:


:DynamicMap   [time,lev]
   :Overlay
      .Land.I      :Feature   [Longitude,Latitude]
      .Image.I     :Image   [lon,lat]   (vmo)
      .Coastline.I :Feature   [Longitude,Latitude]
      .Grid.I      :Feature   [Longitude,Latitude]

## Extract Berring Strait

## Calc Volume Transport